In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import to_categorical
from keras import backend as K
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
from PIL import Image

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os


path = '/home/oziomek/licencjat/kostki/zdjecia/'
listing = os.listdir(path)

num_samples = 0
for i in range(1, 7):
    num_samples += len(os.listdir(path+str(i)))
print(num_samples)

In [ ]:
# for i in range(1, 7):
#     for file in os.listdir(path+str(i)):
#         im = Image.open(path+str(i) + '/' + file)
immatrix = np.array([np.array(Image.open(path+str(i) + '/' + file)).flatten() for i in range(1, 7) for file in os.listdir(path+str(i))], 'f')

In [ ]:
print(immatrix.shape)

In [ ]:
labels = np.ones((num_samples, ), dtype=int)
counter = 0
for i in range(0, 60480, 10080):
    labels[i:i+10080] = counter
    counter +=1

In [ ]:
data, label = shuffle(immatrix, labels, random_state=8)

In [ ]:
rows, cols = 64, 64
labels_values = 6
train_data = [data, label]
img=immatrix[167].reshape(rows, cols)
plt.imshow(img)

In [ ]:
(X, y) = (train_data[0], train_data[1])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 1, rows, cols).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, rows, cols).astype('float32')

# normalize to 0-1
X_train /= 255
X_test /= 255

print(X_train.shape, X_test.shape)

In [ ]:
# onehot
y_train = to_categorical(y_train, 6)
y_test = to_categorical(y_test, 6)

In [ ]:
print(y_train.shape, y_test.shape)

In [ ]:
def base_model():
    model = Sequential()
    model.add(Conv2D(64, (5, 5), input_shape=(1, 64, 64), data_format='channels_first', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(labels_values, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
model = base_model()
print(model.summary())

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=72, verbose=1)

score = model.evaluate(X_test, y_test, verbose=1)

print("Accuracy: %.2f%%" % (scores[1]*100))
print('Test accuracy:', scores[0])
print('Test loss', scores[1])

In [ ]:
model_json = model.to_json()
with open('dice_model.json', 'w') as json_file:
    json_file.write(model_json)
    
model.save_weights('dice_weights.h5')
print('Saved model weights')